In [1]:
import cobra.io

modelo = cobra.io.read_sbml_model("C:\\Users\\lucia\\Desktop\\DielModels\\reconstruction_results\\MODEL1507180028\\results_troppo\\DielModel\\reconstructed_models\\Diel_Model.xml")

In [4]:
for metabolite in modelo.metabolites.query('hv'):
    if "Night" in metabolite.compartment:
        print(metabolite.reactions)

frozenset({<Reaction Ex16_Night at 0x1b5c96385b0>, <Reaction REner01_p_Night at 0x1b5c8b6f7c0>})


In [2]:
from cobra.flux_analysis import pfba

In [3]:
from diel_models.nitrate_uptake_ratio import NitrateUptakeRatioCalibrator

modelo.reactions.get_by_id("Biomass_Total").upper_bound = 0.11
modelo.reactions.get_by_id("Biomass_Total").lower_bound = 0.11
modelo.objective = "Ex16_Day"
modelo.objective_direction = "min"
nitrate_calibrator = NitrateUptakeRatioCalibrator(modelo, "Ex4_Day", "Ex4_Night")
nitrate_calibrator.run()
model = nitrate_calibrator.model
model.reactions.get_by_id("Ex5_Night").bounds = (0, 1000)
model.reactions.get_by_id("Ex5_Day").bounds = (0, 1000)
solution = pfba(modelo)

In [4]:
produced_at_day = solution.fluxes.loc[(solution.fluxes.index.str.contains('Day_sp')) & (round(solution.fluxes, 5) > 0)]

In [9]:
produced_at_night = solution.fluxes.loc[(solution.fluxes.index.str.contains('Night_sp')) & (round(solution.fluxes, 5) < 0)]

In [10]:
import pandas as pd
import os
from cobra.flux_analysis import flux_variability_analysis as fva

fva_solution = fva(modelo, [storage for storage in modelo.reactions if "_sp" in storage.id], fraction_of_optimum=1.0, processes=os.cpu_count())
produced_at_day = pd.concat([produced_at_day, fva_solution.loc[produced_at_day.index, ['minimum', 'maximum']]],
                                axis=1)
produced_at_night = pd.concat([produced_at_night, fva_solution.loc[produced_at_night.index, ['minimum', 'maximum']]], axis=1)

In [11]:
produced_at_day

,fluxes,minimum,maximum,minimum,maximum
Sucrose_c_Day_sp_exchange,0.024970,-0.296093,0.296093,-0.296093,0.296093
L-Isoleucine_Day_sp_exchange,0.003300,-0.003300,0.003300,-0.003300,0.003300
L-Leucine_Day_sp_exchange,0.007700,-0.007700,0.007700,-0.007700,0.007700
L-Lysine_Day_sp_exchange,0.000055,-0.000055,0.000055,-0.000055,0.000055
L-Methionine_Day_sp_exchange,0.000110,-0.000110,0.000110,-0.000110,0.000110
L-Phenylalanine_Day_sp_exchange,0.004400,-0.027940,0.027940,-0.027940,0.027940
L-Threonine_Day_sp_exchange,0.004730,-0.011550,0.011550,-0.011550,0.011550
L-Tryptophan_Day_sp_exchange,0.002200,-0.002200,0.002200,-0.002200,0.002200
L-Cysteine_Day_sp_exchange,0.001320,-0.001320,0.001320,-0.001320,0.001320
L-Glutamine_c_Day_sp_exchange,0.140503,-0.481077,0.481077,-0.481077,0.481077


In [12]:
produced_at_night

,fluxes,minimum,maximum
Nitrate_c_Night_sp_exchange,-0.137528,-0.140184,0.210276
L-Valine_Night_sp_exchange,-0.020900,-0.020900,0.020900
L-Proline_Night_sp_exchange,-0.000550,-0.267658,3.966457
L-Alanine_c_Night_sp_exchange,-1.540705,-9.637505,-1.458893
D-Fructose_Night_sp_exchange,-0.033165,-0.559020,0.559020
Citrate_c_Night_sp_exchange,-1.514663,-9.747674,-1.348725


### Reação Fotossíntese

In [10]:
Rener01_p_Day_flux_diel_model = solution.fluxes['REner01_p_Day']
Rener01_p_Day_flux_diel_model

125.0

In [11]:
Rener01_p_Night_flux_diel_model = solution.fluxes['REner01_p_Night']
Rener01_p_Night_flux_diel_model

0.0

### Reações Ciclo de Calvin

- Rubisco + CO2

In [15]:
modelo.reactions.get_by_id('R00024_p_Night')

Reaction identifier,R00024_p_Night
Name,ribulose bisphosphate carboxylase small chain 1A / RuBisCO small subunit 1A (RBCS-1A) (ATS1A)
Memory address,0x226f55dc880
Stoichiometry,"S_CO2_p[C_p]_Night + S_D_45_Ribulose_32_1_44_5_45_bisphosphate_p[C_p]_Night + S_H2O_p[C_p]_Night --> 2.0 S_3_45_Phospho_45_D_45_glycerate_p[C_p]_Night CO2_p Night + D-Ribulose 1,5-bisphosphate Night + H2O_p Night --> 2.0 3-Phospho-D-glycerate_p Night"
GPR,
Lower bound,0.0
Upper bound,1000.0


In [21]:
R00024_p_Night_flux_diel_model = solution.fluxes['R00024_p_Night']
R00024_p_Night_flux_diel_model

0.3374561054911554

In [22]:
R00024_p_Day_flux_diel_model = solution.fluxes['R00024_p_Day']
R00024_p_Day_flux_diel_model

106.77663763966915

- Rubisco + Oxigénio

In [32]:
modelo.reactions.get_by_id('R03140_p_Day')

Reaction identifier,R03140_p_Day
Name,ribulose bisphosphate carboxylase small chain 1A / RuBisCO small subunit 1A (RBCS-1A) (ATS1A)
Memory address,0x1b5c8f26ac0
Stoichiometry,"S_D_45_Ribulose_32_1_44_5_45_bisphosphate_p[C_p]_Day + S_Oxygen_p[C_p]_Day --> S_2_45_Phosphoglycolate_p[C_p]_Day + S_3_45_Phospho_45_D_45_glycerate_p[C_p]_Day D-Ribulose 1,5-bisphosphate Day + Oxygen_p Day --> 2-Phosphoglycolate Day + 3-Phospho-D-glycerate_p Day"
GPR,ARTHCP030 or AT1G67090 or AT5G38410 or AT5G38420 or AT5G38430
Lower bound,0.0
Upper bound,1000.0


In [31]:
modelo.reactions.get_by_id('R03140_p_Night')

Reaction identifier,R03140_p_Night
Name,ribulose bisphosphate carboxylase small chain 1A / RuBisCO small subunit 1A (RBCS-1A) (ATS1A)
Memory address,0x1b5c987bfa0
Stoichiometry,"S_D_45_Ribulose_32_1_44_5_45_bisphosphate_p[C_p]_Night + S_Oxygen_p[C_p]_Night --> S_2_45_Phosphoglycolate_p[C_p]_Night + S_3_45_Phospho_45_D_45_glycerate_p[C_p]_Night D-Ribulose 1,5-bisphosphate Night + Oxygen_p Night --> 2-Phosphoglycolate Night + 3-Phospho-D-glycerate_p Night"
GPR,
Lower bound,0.0
Upper bound,1000.0


In [36]:
R03140_p_Day_flux_diel_model = solution.fluxes['R03140_p_Day']

In [37]:
R03140_p_Night_flux_diel_model = solution.fluxes['R03140_p_Night']

- PGK

In [54]:
modelo.reactions.get_by_id("R01512_c_Day")

Reaction identifier,R01512_c_Day
Name,"phosphoglycerate kinase, putative"
Memory address,0x1b5c8d5bca0
Stoichiometry,S_3_45_Phospho_45_D_45_glycerate_c[C_c]_Day + S_ATP_c[C_c]_Day <=> S_3_45_Phospho_45_D_45_glyceroyl_32_phosphate_c[C_c]_Day + S_ADP_c[C_c]_Day 3-Phospho-D-glycerate_c Day + ATP_c Day <=> 3-Phospho-D-glyceroyl phosphate Day + ADP_c Day
GPR,AT1G56190 or AT1G79550 or AT3G12780
Lower bound,-1000.0
Upper bound,1000.0


In [55]:
modelo.reactions.get_by_id("R01512_c_Night")

Reaction identifier,R01512_c_Night
Name,"phosphoglycerate kinase, putative"
Memory address,0x1b5c978cf70
Stoichiometry,S_3_45_Phospho_45_D_45_glycerate_c[C_c]_Night + S_ATP_c[C_c]_Night <=> S_3_45_Phospho_45_D_45_glyceroyl_32_phosphate_c[C_c]_Night + S_ADP_c[C_c]_Night 3-Phospho-D-glycerate_c Night + ATP_c Night <=> 3-Phospho-D-glyceroyl phosphate Night + ADP_c Night
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [45]:
R01512_c_Day_flux_diel_model = solution.fluxes['R01512_c_Day']

In [44]:
R01512_c_Night_flux_diel_model =  solution.fluxes['R01512_c_Night']

- Frutose 1,6 bifostato -> Frutose 1,6 fosfato

In [51]:
modelo.reactions.get_by_id("R04780_c_Day")

Reaction identifier,R04780_c_Day
Name,"fructose-1,6-bisphosphatase, putative / D-fructose-1,6-bisphosphate 1-phosphohydrolase, putative..."
Memory address,0x1b5c90b6fa0
Stoichiometry,"S_H2O_c[C_c]_Day + S_beta_45_D_45_Fructose_32_1_44_6_45_bisphosphate_c[C_c]_Day --> S_Orthophosphate_c[C_c]_Day + S_beta_45_D_45_Fructose_32_6_45_phosphate_c[C_c]_Day H2O_c Day + beta-D-Fructose 1,6-bisphosphate Day --> Orthophosphate_c Day + beta-D-Fructose 6-phosphate Day"
GPR,AT1G43670 or AT3G54050
Lower bound,0.0
Upper bound,1000.0


In [52]:
R04780_c_Day_flux_diel_model = solution.fluxes['R04780_c_Day']

In [53]:
R04780_c_Night_flux_diel_model = solution.fluxes['R04780_c_Night']

In [67]:
modelo.metabolites.get_by_id("S_D_45_Ribulose_32_1_44_5_45_bisphosphate_p[C_p]_Night").reactions

frozenset({<Reaction R00024_p_Night at 0x1b5c966dcd0>,
           <Reaction R01523_p_Night at 0x1b5c97979d0>,
           <Reaction R03140_p_Night at 0x1b5c987bfa0>})

In [42]:
modelo.reactions.get_by_id("R00762_p_Night")

Reaction identifier,R00762_p_Night
Name,"fructose-1,6-bisphosphatase, putative / D-fructose-1,6-bisphosphate 1-phosphohydrolase, putative..."
Memory address,0x226f5671bb0
Stoichiometry,"S_D_45_Fructose_32_1_44_6_45_bisphosphate_p[C_p]_Night + S_H2O_p[C_p]_Night --> S_D_45_Fructose_32_6_45_phosphate_p[C_p]_Night + S_Orthophosphate_p[C_p]_Night D-Fructose 1,6-bisphosphate Night + H2O_p Night --> D-Fructose 6-phosphate Night + Orthophosphate_p Night"
GPR,
Lower bound,0.0
Upper bound,1000.0


In [60]:
modelo.reactions.get_by_id("R01529_c_Day")

Reaction identifier,R01529_c_Day
Name,"ribulose-phosphate 3-epimerase, cytosolic, putative / pentose-5-phosphate 3-epimerase, putative"
Memory address,0x1b5c8d7bee0
Stoichiometry,S_D_45_Ribulose_32_5_45_phosphate_c[C_c]_Day <=> S_D_45_Xylulose_32_5_45_phosphate_c[C_c]_Day D-Ribulose 5-phosphate Day <=> D-Xylulose 5-phosphate Day
GPR,AT1G63290 or AT3G01850 or AT5G61410
Lower bound,-1000.0
Upper bound,1000.0


In [61]:
R01529_c_Day_flux_diel_model = solution.fluxes["R01529_c_Day"]

In [62]:
R01529_c_Night_flux_diel_model = solution.fluxes["R01529_c_Night"]

In [13]:
from tests import TEST_DIR
import os
original_model = cobra.io.read_sbml_model(os.path.join(TEST_DIR, 'data', 'aragem_photo.xml'))

In [16]:
original_model.reactions.get_by_id("REner01_p")

Reaction identifier,REner01_p
Name,R_REner01_p
Memory address,0x1b5cc17c280
Stoichiometry,3.0 S_ADP_p[C_p] + 2.0 S_H2O_p[C_p] + 2.0 S_NADP_43__p[C_p] + 3.0 S_Orthophosphate_p[C_p] + 8.0 S_hv_p[C_p] --> 3.0 S_ATP_p[C_p] + 2.0 S_H_43__p[C_p] + 2.0 S_NADPH_p[C_p] + S_Oxygen_p[C_p] 3.0 ADP_p + 2.0 H2O_p + 2.0 NADP+ + 3.0 Orthophosphate_p + 8.0 hv_p --> 3.0 ATP_p + 2.0 H+_p + 2.0 NADPH + Oxygen_p
GPR,
Lower bound,0.0
Upper bound,1000.0


In [18]:
original_model.objective_direction = "max"
solution_original = pfba(original_model)

In [19]:
Rener01_p_flux_original_model = solution_original.fluxes["REner01_p"]

In [20]:
Rener01_p_flux_original_model

125.0

In [23]:
R00024_p_flux_diel_model = solution_original.fluxes["R00024_p"]

In [34]:
R03140_p_flux_diel_model = solution_original.fluxes['R03140_p']

In [47]:
R01512_c_flux_diel_model = solution_original.fluxes['R01512_c']

In [57]:
R04780_c_flux_diel_model = solution_original.fluxes['R04780_c']

In [64]:
R01529_c_flux_diel_model = solution_original.fluxes["R01529_c"]

In [73]:
import pandas as pd
import matplotlib.pyplot as plt

data = {'Photosynthesis Reaction': [Rener01_p_Day_flux_diel_model, Rener01_p_Night_flux_diel_model,
                                                Rener01_p_flux_original_model],
        'RuBisCO + CO2': [R00024_p_Day_flux_diel_model, R00024_p_Night_flux_diel_model, R00024_p_flux_diel_model],
        'RuBisCO + O2' : [R03140_p_Day_flux_diel_model, R03140_p_Night_flux_diel_model, R03140_p_flux_diel_model],
        'Fructose Reaction' : [R04780_c_Day_flux_diel_model, R04780_c_Night_flux_diel_model, R04780_c_flux_diel_model],
        'Xylulose-Ribulose Reaction' : [R01529_c_Day_flux_diel_model, R01529_c_Night_flux_diel_model, R01529_c_flux_diel_model]
        }

df = pd.DataFrame(data)

new_index = ['Diel Model Day', 'Diel Model Night', 'Original Model']

for index, row in df.iterrows():
    for col in df.columns:
        df.at[index, col] = round(df.at[index, col], 3)

df.set_index(pd.Index(new_index), inplace=True)

df

,Photosynthesis Reaction,RuBisCO + CO2,RuBisCO + O2,Fructose Reaction,Xylulose-Ribulose Reaction
Diel Model Day,125.0,106.777,0.000,1.068,-0.402
Diel Model Night,0.0,0.337,0.000,0.000,0.187
Original Model,125.0,2.473,0.035,0.057,-0.001
